In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai import AzureOpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter
load_dotenv()
print(load_dotenv())

os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")



True


## **Custom agent**

**Define the functions**

In [2]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [3]:
abc(2, 3)

-2

In [4]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [5]:
abc_json = jsonschema(abc)
abc_json

{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [6]:
model_name = "gpt4o"

**Ask GPT**

In [8]:
client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)
response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [92]:
response.choices[0].message.content

'Could you clarify what you mean by "abc"? Are you referring to an arithmetic operation, an acronym, or something else?'

In [93]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

In [94]:
response

ChatCompletion(id='chatcmpl-AIyGGa5TfaDwnRwp8FlsmCHgxxwaM', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"num1":2,"num2":3}', name='abc'), tool_calls=None), content_filter_results={})], created=1729084912, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_67802d9a6d', usage=CompletionUsage(completion_tokens=20, prompt_tokens=69, total_tokens=89), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

**Executing the function by extracting the info from the output of the model**

In [95]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{"num1":2,"num2":3}', name='abc')
{"num1":2,"num2":3}
<class 'str'>


In [96]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [97]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


## **Using Langchain**

In [98]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [99]:
abc.description

'Compute abc between two numbers'

In [100]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"))

In [104]:
tools = [abc]

llm_with_tools = llm.bind_tools(tools)
print(f"Using API Key: {os.getenv('AZURE_OPENAI_API_KEY')}")


Using API Key: EnvrvVgQgVNjzHrXY57TUyESTNeIqovna3WsB7mctCSmyrICEJ2lJQQJ99AJACHYHv6XJ3w3AAABACOG2fTJ


In [105]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: EnvrvVgQ************************************************************************2fTJ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [320]:
response.additional_kwargs

AttributeError: 'ChatCompletion' object has no attribute 'additional_kwargs'